# Install Python Dependencies from `require.txt`

In [1]:
pip install -r "require.txt"

  Using cached cython-3.1.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached tabula_py-2.10.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached ghostscript-0.7-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached tensorflow-2.19.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.0 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0)
ERROR: No matching distribution found for torch==2.0.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Initialize SentenceTransformer Model (`all-MiniLM-L12-v2`)

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L12-v2")
print("✅ SentenceTransformer imported successfully!")

✅ SentenceTransformer imported successfully!


# Locate a Specific File by Name in a Folder and Subfolders

In [3]:
import os

filename = "file_example_XLS_5000 (2).xls"  # Change to your actual file name
search_path = "/Users/shabil/Desktop"  # Change to your search directory

for root, dirs, files in os.walk(search_path):
    if filename in files:
        file_path = os.path.join(root, filename)
        print("✅ File found at:", file_path)
        break
else:
    print("❌ File not found!")

✅ File found at: /Users/shabil/Desktop/file_example_XLS_5000 (2).xls


# Function to Load Excel Files (.xls or .xlsx) and Preview Data

In [4]:
import pandas as pd

# Load Excel File
def load_excel(file_path):
    try:
        # Choose correct engine based on file type
        if file_path.endswith(".xls"):
            df = pd.read_excel(file_path, engine="xlrd")  # For .xls
        else:
            df = pd.read_excel(file_path, engine="openpyxl")  # For .xlsx
        
        print("✅ Successfully loaded file!")
        print("📌 Columns detected:", df.columns.tolist())  # Show column names

        # Display all rows and columns
        pd.set_option("display.max_rows", None)  # Show all rows
        pd.set_option("display.max_columns", None)  # Show all columns
        pd.set_option("display.expand_frame_repr", False)  # Prevent line breaks

        return df
    except Exception as e:
        print("❌ Error loading file:", e)
        return None

# Example Usage:
file_path = "/Users/shabil/Desktop/file_example_XLS_5000 (2).xls"  # Ensure correct path
df = load_excel(file_path)

if df is not None and not df.empty:
    print(df.head())  # Show first few rows for verification
else:
    print("⚠️ Could not load the Excel file. Check the error above.")

✅ Successfully loaded file!
📌 Columns detected: ['Unnamed: 0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date', 'Id']
   Unnamed: 0 First Name  Last Name  Gender        Country  Age        Date    Id
0           1      Dulce      Abril  Female  United States   32  15/10/2017  1562
1           2       Mara  Hashimoto  Female  Great Britain   25  16/08/2016  1582
2           3     Philip       Gent    Male         France   36  21/05/2015  2587
3           4   Kathleen     Hanner  Female  United States   25  15/10/2017  3549
4           5    Nereida    Magwood  Female  United States   58  16/08/2016  2468


In [5]:
# Clean Data
def clean_data(df):
    if df is None or df.empty:
        print("⚠️ DataFrame is empty. Skipping cleaning.")
        return None
    
    print("🛠 Cleaning Data...")

    # Drop duplicate rows
    df = df.drop_duplicates()

    # Remove rows with all NaN values
    df = df.dropna(how="all")

    # Trim whitespace from string columns
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Fill missing values (example: replacing NaN with 'Unknown' in object columns)
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].fillna("Unknown")

    print("Data cleaning complete!")
    print("Data shape after cleaning:", df.shape)  
    return df

# Apply cleaning
df = clean_data(df)

🛠 Cleaning Data...
Data cleaning complete!
Data shape after cleaning: (5000, 8)


/var/folders/0s/3sj3r3sx07b3fgdmcbmgjcy00000gn/T/ipykernel_14293/4122198992.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Data Cleaning Function

# Step 1: Initialize Dependencies and Memory

In [6]:
import ollama
import chromadb
from sentence_transformers import SentenceTransformer
import json
from datetime import datetime
from typing import List, Optional

# Simplified approach without complex LangChain chains
class SimplifiedLangChainChatbot:
    def __init__(self):
        # Initialize ChromaDB
        self.client = chromadb.PersistentClient(path="./vector_db")  
        self.collection = self.client.get_or_create_collection("company_info")
        
        # Initialize SentenceTransformer
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        
        # Simple conversation memory
        self.conversation_history = []

#  Step2:Vector Search and User Guidance Helpers

In [7]:
    def search_vector_db(self, query, top_k=50):
        """Search the vector database"""
        query_embedding = self.model.encode(query).tolist()
        results = self.collection.query(query_embeddings=[query_embedding], n_results=top_k)
        if results and "documents" in results and results["documents"]:
            retrieved_texts = [doc for sublist in results["documents"] for doc in sublist]
            return list(set(retrieved_texts))  # Ensure unique entries only
        return []
    
    def get_current_datetime(self):
        """Get current date and time"""
        now = datetime.now()
        return now.strftime("%A, %B %d, %Y at %I:%M %p")
    
    def suggest_questions(self):
        """Suggest example questions"""
        examples = [
            "What is the total revenue for the last quarter?",
            "How many employees are working in the IT department?",
            "Can you list the top 5 performing products?",
            "What was the highest sale recorded this year?",
            "Provide a summary of all available data."
        ]
        return "\n".join(f"{i+1}. {q}" for i, q in enumerate(examples))

# Step 3: Handle Queries with Context and AI Response

In [8]:
    def build_context_with_history(self, current_query, retrieved_data):
        """Build context including conversation history"""
        context_parts = []
        
        # Add conversation history (last 3 exchanges)
        if self.conversation_history:
            context_parts.append("### Recent Conversation:")
            for i, (q, a) in enumerate(self.conversation_history[-3:]):
                context_parts.append(f"Previous Q{i+1}: {q}")
                context_parts.append(f"Previous A{i+1}: {a[:200]}...")  # Truncate long answers
        
        # Add current retrieved data
        if retrieved_data:
            context_parts.append("### Current Dataset Context:")
            context_parts.extend(retrieved_data)
        
        return "\n\n".join(context_parts)
    
    def chat_with_ollama(self, query, retrieved_data, date_info):
        """Enhanced chat with Ollama including conversation context"""
        
        if not retrieved_data:
            example_questions = self.suggest_questions()
            return f"I couldn't find any relevant data. Please ask only about the provided dataset.\n\nYou can ask questions for example:\n{example_questions}"
        
        # Build context with conversation history
        full_context = self.build_context_with_history(query, retrieved_data)
        
        full_prompt = f"""You are an AI assistant analyzing a dataset with conversation memory.
Answer **only** using the data provided. If a user asks about something outside the dataset, tell them to ask about the provided information.

{date_info}

{full_context}

### Current User Query:
{query}

**Instructions:**
- Consider the conversation history when answering
- Do **not summarize excessively**—include all relevant information
- If the user asks for a list, **format the response properly**
- If the answer is too long, split it into **clear sections**
- If you are unsure, politely say: "I can only answer questions related to the dataset provided."

Answer:"""
        
        try:
            response = ollama.chat(
                model="mistral",  
                messages=[{"role": "user", "content": full_prompt}],
                options={"num_predict": 16384}
            )
            answer = response["message"]["content"]
            
            # Store in conversation history
            self.conversation_history.append((query, answer))
            
            # Keep only last 5 conversations to manage memory
            if len(self.conversation_history) > 5:
                self.conversation_history = self.conversation_history[-5:]
            
            return answer
            
        except Exception as e:
            return f"Error communicating with Ollama: {str(e)}"
    
    def process_query(self, query: str) -> str:
        """Process a user query"""
        current_datetime = self.get_current_datetime()
        date_message = f"Today's date and time is {current_datetime}."
        
        # Special handling for listing people
        if "list all the people" in query.lower():
            retrieved_data = self.search_vector_db(query, top_k=50)
            if retrieved_data:
                formatted_people = "\n".join([f"- {data}" for data in retrieved_data])
                answer = f"Here are the unique people found in the dataset:\n\n{formatted_people}"
                self.conversation_history.append((query, answer))
                return answer
            else:
                return "I couldn't find any relevant data about people in the dataset."
        
        # Regular query processing
        retrieved_data = self.search_vector_db(query)
        return self.chat_with_ollama(query, retrieved_data, date_message)

# Step 3: Maintain Dataset and Conversation History

In [9]:
    def add_documents_to_vector_store(self, texts: List[str], metadatas: Optional[List[dict]] = None):
        """Add documents to the vector store"""
        embeddings = [self.model.encode(text).tolist() for text in texts]
        ids = [f"doc_{i}_{datetime.now().timestamp()}" for i in range(len(texts))]
        
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas or [{"source": f"added_doc_{i}"} for i in range(len(texts))],
            ids=ids
        )
        print(f"✅ Added {len(texts)} documents to vector store")
    
    def clear_conversation_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        print("🧹 Conversation history cleared!")
    
    def show_conversation_history(self):
        """Show recent conversation history"""
        if not self.conversation_history:
            print("No conversation history available.")
            return
        
        print("\n📜 Recent Conversation History:")
        for i, (q, a) in enumerate(self.conversation_history[-3:], 1):
            print(f"\n{i}. Q: {q}")
            print(f"   A: {a[:100]}...")

# Step4: Main Interactive Chat Loop for User Queries and Commands

In [10]:
    def chat_loop(self):
        """Main chat loop"""
        print("🤖 Enhanced LangChain-Style Chatbot initialized!")
        print("You can now ask questions about your dataset.")
        print("\nCommands:")
        print("- 'exit' to quit")
        print("- 'clear' to clear conversation history")
        print("- 'history' to show recent conversation")
        print("- 'help' for example questions")
        
        while True:
            user_query = input("\nAsk about any details: ").strip()
            
            if not user_query:
                continue
                
            if user_query.lower() == "exit":
                print("Goodbye! 👋")
                break
            elif user_query.lower() == "clear":
                self.clear_conversation_history()
                continue
            elif user_query.lower() == "history":
                self.show_conversation_history()
                continue
            elif user_query.lower() == "help":
                example_questions = self.suggest_questions()
                print(f"\nHere are some example questions you can ask:\n{example_questions}")
                continue
            
            # Process the query
            print("\n🤖 Thinking...")
            response = self.process_query(user_query)
            print(f"\n🤖 Chat:\n{response}")


# Step5:Main Function: Initialize and Start Chatbot

In [11]:
# Usage
def main():
    # Initialize the chatbot
    chatbot = SimplifiedLangChainChatbot()
    chatbot.chat_loop()

if __name__ == "__main__":
    main()

AttributeError: 'SimplifiedLangChainChatbot' object has no attribute 'chat_loop'